# driver_monolithic_NMROM
Driver to implement and test NM ROM on the 2D Burgers Equation.  

In [ ]:
import sys
sys.path.append("/Users/zanardi1/Workspace/Codes/DD-NM-ROM/dd-nm-rom")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from matplotlib import cm
import matplotlib.colors as colors
import scipy.sparse as sp
import scipy.linalg as la
from time import time
import dill as pickle
import os
from utils.Burgers2D_probgen import Burgers2D
from utils.domain_decomposition import DD_model
from utils.monolithic_LSROM import compute_basis_from_svd #, select_sample_nodes
# from utils.NM_ROM import DD_NM_ROM, separate_snapshots, compute_residual_bases, RBFmdl, get_net_np_params, sp_diag
from utils.monolithic_NMROM import monolithic_NMROM, monolithic_NMROM_HR
from utils.newton_solve import gauss_newton
from scipy.interpolate import RBFInterpolator

In [ ]:
data_dir = './data/'
fig_dir0 = './figures/'
for d in [data_dir, fig_dir0]:
    if not os.path.exists(d): os.mkdir(d)
plt.rc('font', size=20)
plt.rcParams['text.usetex'] = True

## Set model parameters

In [ ]:
# define constant parameters for PDE
nx, ny  = 480, 24
x_lim   = [-1.0, 1.0]
y_lim   = [0.0, 0.05]

na1, nlam = 80, 80
a1_lim  = [1.0, 10000.0]
lam_lim = [5.0, 25.0]

a1, lam = 7692.5384, 21.9230
viscosity = 1e-1

print(f'a1, lam = {a1, lam}')

# number of subdomains in x and y directions for DD model
n_sub_x = 1
n_sub_y = 1

n_sub   = n_sub_x*n_sub_y
fig_dir1 = fig_dir0 + f'nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y/'
fig_dir  = fig_dir1 + f'a1_{a1}_lam_{lam}/'
svd_dir  = data_dir + f'nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y/'
nm_figs  = fig_dir + 'nmrom/'
for d in [fig_dir1, fig_dir, nm_figs]:
    if not os.path.exists(d):
        os.mkdir(d)
if not os.path.exists(svd_dir): print('Error: Must run LS-ROM notebook before proceeding.')

In [ ]:
# load snapshot and residual data
file = f'./data/residual_nx_{nx}_ny_{ny}_mu_{viscosity}_Nsamples_{400}.p'
data = pickle.load(open(file, 'rb'))
Mu = data['parameters']
snapshots = data['snapshots']
residuals = data['residuals']

## Solve monolithic FOM

In [ ]:
# compute FOM for given a1 and lambda
def u_exact(x, y):
    phi = a1 + a1*x + (np.exp(lam*(x-1.0)) + np.exp(-lam*(x - 1.0)))*np.cos(lam*y)
    val = -2.0*viscosity*(a1 + lam*(np.exp(lam*(x-1.0)) \
          - np.exp(-lam*(x - 1.0)))*np.cos(lam*y))/phi
    return val
def v_exact(x, y):
    phi = a1 + a1*x + (np.exp(lam*(x-1.0)) + np.exp(-lam*(x - 1.0)))*np.cos(lam*y)
    val =  2.0*viscosity*(lam*(np.exp(lam*(x-1.0)) \
           + np.exp(-lam*(x - 1.0)))*np.sin(lam*y))/phi
    return val

# generate Burgers FOM on full domain
fom = Burgers2D(nx, ny, x_lim, y_lim, viscosity, u_exact, v_exact)
print('Solving monolithic FOM:')
start = time()
u_fom, v_fom, res_hist = fom.solve(np.zeros(fom.nxy), np.zeros(fom.nxy), tol=1e-8, print_hist=True)
fom_time = time()-start
sol_fd = np.concatenate([u_fom, v_fom])

In [ ]:
# plot DD FOM u and v
x = np.linspace(x_lim[0], x_lim[1], nx+2)[1:-1]
y = np.linspace(y_lim[0], y_lim[1], ny+2)[1:-1]
X, Y = np.meshgrid(x, y)
U_fom = u_fom.reshape(ny, nx)
V_fom = v_fom.reshape(ny, nx)

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, U_fom, cmap=cm.jet, shading='auto', vmin=u_fom.min(), vmax=u_fom.max())
# plt.xlabel('$x$')
# plt.ylabel('$y$')
cb = plt.colorbar()#label='$u(x, y)$')
file = fig_dir+'u_fom.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, V_fom, cmap=cm.jet, shading='auto', vmin=v_fom.min(), vmax=v_fom.max())
# plt.xlabel('$x$')
# plt.ylabel('$y$')
cb = plt.colorbar()#label='$v(x, y)$')
file = fig_dir+'v_fom.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Generate NM-ROM class

In [ ]:
ec     = 1e-10          # energy criteria for number of interior and interface basis vectors
nbasis = -1             # number of residual basis vectors. set to -1 to use energy criteria

# load SVD data
print('Loading residual SVD data...')
res_dict  = pickle.load(open(svd_dir+'res_svd.p', 'rb'))
print('Data loaded!')

# compute bases
print('Computing residual basis...')
residual_basis = compute_basis_from_svd(res_dict, ec=ec, nbasis=nbasis)
print('Bases computed!')

print(f'residual_basis.shape={residual_basis.shape}')

In [ ]:
# autoencoder parameters
latent_dim = 9
row_nnz    = 5
row_shift  = 5
act_type   = 'Swish'

Ntotal = 6400 if nx in [240, 480] else 4200
net_folder0 = f'./trained_nets/nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y/sub_1of1/'
net_name = net_folder0+\
           f'ld_{latent_dim}_rnnz_{row_nnz}_rshift_{row_shift}_'+\
           f'{act_type}_batch_32_AbsMSEloss_{Ntotal}snaps.p' 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net_dict = torch.load(net_name, map_location=device)

In [ ]:
# instatiate NM-ROM
nmrom = monolithic_NMROM(fom, net_dict)

In [ ]:
# train RBF interpolator for initial iterate
enc_snapshots = np.vstack([nmrom.encoder(s) for s in snapshots])
rbfmdl = RBFInterpolator(Mu, enc_snapshots, neighbors=None, kernel='linear')

In [ ]:
# solve nmrom
start = time()
w0 = rbfmdl(np.array([[a1,lam]])).reshape(-1,) # compute initial iterate
w, res_hist, step_hist, iter = nmrom.solve(w0, tol=1e-3, maxit=5, print_hist=True)
rom_time = time()-start
nmrom_rel_err = np.linalg.norm(sol_fd-nmrom.decoder(w)[0])/np.linalg.norm(sol_fd)
print(f'DD-ROM rel. error = {nmrom_rel_err:1.4e}')
print(f'Speedup = {fom_time/rom_time}')

In [ ]:
# plot rom solutions
uv_rom = nmrom.decoder(w)[0]
u_rom = uv_rom[:nmrom.nxy]
v_rom = uv_rom[nmrom.nxy:]
U_rom = u_rom.reshape(ny, nx)
V_rom = v_rom.reshape(ny, nx)

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, U_rom, cmap=cm.jet, shading='auto', vmin=u_fom.min(), vmax=u_fom.max())
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar()#label='$u(x, y)$')
file = nm_figs+'u_state.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, V_rom, cmap=cm.jet, shading='auto', vmin=v_fom.min(), vmax=v_fom.max())
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar()#label='$v(x, y)$')
file = nm_figs+'v_state.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# plot errors
u_rel_err = np.abs(U_fom-U_rom)/np.linalg.norm(U_fom)
plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, u_rel_err, cmap=cm.jet, shading='auto')#, vmin=0.0, vmax=4.88e-4)
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar()#label='$u$ Relative error')
file = nm_figs+f'u_rel_error.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12, 4))
v_rel_err = np.abs(V_fom-V_rom)/np.linalg.norm(V_fom)
plt.pcolormesh(X, Y, v_rel_err, cmap=cm.jet, shading='auto')#, vmin=0.0, vmax=4.88e-4)
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar()#label='$v$ Relative error')
file = nm_figs+f'v_rel_error.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

## NMROM with HR

In [ ]:
# instantiate NMROM with HR
nmromhr = monolithic_NMROM_HR(fom, net_dict, residual_basis, sample_ratio=2, n_samples=-1)

In [ ]:
# train RBF interpolator for initial iterate
enc_snapshots = np.vstack([nmromhr.encoder(s) for s in snapshots])
rbfmdl = RBFInterpolator(Mu, enc_snapshots, neighbors=None, kernel='linear')

In [ ]:
# solve nmrom
start = time()
w0 = rbfmdl(np.array([[a1,lam]])).reshape(-1,) # compute initial iterate
w_hr, res_hist, step_hist, iter = nmromhr.solve(w0, tol=1e-3, maxit=5, print_hist=True)
hr_rom_time = time()-start
nmromhr_rel_err = np.linalg.norm(sol_fd-nmromhr.decoder(w_hr)[0])/np.linalg.norm(sol_fd)
print(f'DD-ROM rel. error = {nmromhr_rel_err:1.4e}')
print(f'Speedup = {fom_time/hr_rom_time}')

In [ ]:
# plot rom solutions
uv_rom = nmromhr.decoder(w_hr)[0]
u_rom = uv_rom[:nmromhr.nxy]
v_rom = uv_rom[nmromhr.nxy:]
U_rom = u_rom.reshape(ny, nx)
V_rom = v_rom.reshape(ny, nx)

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, U_rom, cmap=cm.jet, shading='auto', vmin=u_fom.min(), vmax=u_fom.max())
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar()#label='$u(x, y)$')
file = nm_figs+'u_col_hr_state.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, V_rom, cmap=cm.jet, shading='auto', vmin=v_fom.min(), vmax=v_fom.max())
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar()#label='$v(x, y)$')
file = nm_figs+'v_col_hr_state.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
# plot errors
u_rel_err = np.abs(U_fom-U_rom)/np.linalg.norm(U_fom)
plt.figure(figsize=(12, 4))
plt.pcolormesh(X, Y, u_rel_err, cmap=cm.jet, shading='auto')#, vmin=0.0, vmax=4.88e-4)
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar()#label='$u$ Relative error')
file = nm_figs+f'u_col_hr_rel_error.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

plt.figure(figsize=(12, 4))
v_rel_err = np.abs(V_fom-V_rom)/np.linalg.norm(V_fom)
plt.pcolormesh(X, Y, v_rel_err, cmap=cm.jet, shading='auto')#, vmin=0.0, vmax=4.88e-4)
plt.xlabel('$x$')
plt.ylabel('$y$')
cb = plt.colorbar()#label='$v$ Relative error')
file = nm_figs+f'v_col_hr_rel_error.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
n_params = nmrom.en_W1.nnz + nmrom.en_b1.size + np.prod(nmrom.en_W2.shape) \
           + nmrom.de_W2.nnz + nmrom.de_b1.size + np.prod(nmrom.de_W1.shape)

In [ ]:
nmromhr.train_time

In [ ]:
nmromhr.train_time/1276.4079313278198

In [ ]:
n_params

In [ ]:
P = [1.147e6, 5.257e5, 2.617e5, 1.297e5]
red = [(1-p/n_params)*100 for p in P]
red

In [ ]:
[p/3.226e6 for p in P]